In [12]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
df = pd.read_csv("insurance_data.csv")
df.head()

age  affordibility  bought_insurance
0   22              1                 0
1   25              0                 0
2   47              1                 1
3   52              0                 0
4   46              1                 1

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2,random_state=25)

In [17]:
len(x_train)

22

In [18]:
x_train_scaled = x_train.copy()
x_train_scaled['age'] = x_train_scaled['age']/100

x_test_scaled = x_test.copy()
x_test_scaled['age'] = x_test_scaled['age']/100

In [21]:
model = keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']  
)

In [22]:
model.fit(x_train_scaled,y_train,epochs=5000)

Epoch 1/5000
1/1 [==============================] - 1s 635ms/step - loss: 0.7113 - accuracy: 0.5000
Epoch 2/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.7110 - accuracy: 0.5000
Epoch 3/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.7106 - accuracy: 0.5000
Epoch 4/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.7102 - accuracy: 0.5000
Epoch 5/5000
1/1 [==============================] - 0s 13ms/step - loss: 0.7098 - accuracy: 0.5000
Epoch 6/5000
1/1 [==============================] - 0s 23ms/step - loss: 0.7094 - accuracy: 0.5000
Epoch 7/5000
1/1 [==============================] - 0s 17ms/step - loss: 0.7091 - accuracy: 0.5000
Epoch 8/5000
1/1 [==============================] - 0s 16ms/step - loss: 0.7087 - accuracy: 0.5000
Epoch 9/5000
1/1 [==============================] - 0s 16ms/step - loss: 0.7083 - accuracy: 0.5000
Epoch 10/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.7079 - accuracy: 0.5000
Epoch 11/50

In [24]:
model.evaluate(x_test_scaled,y_test)

1/1 [==============================] - 0s 26ms/step - loss: 0.3550 - accuracy: 1.0000


[0.35497748851776123, 1.0]

In [26]:
model.predict(x_test_scaled)

1/1 [==============================] - 0s 22ms/step


array([[0.7054848 ],
       [0.35569543],
       [0.16827849],
       [0.47801173],
       [0.72606975],
       [0.8294984 ]], dtype=float32)

In [39]:
np.transpose(x_train_scaled['age']).shape

(22,)

In [30]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [41]:
coef , intercept = model.get_weights()
coef , intercept

(array([[5.060868],
        [1.40865 ]], dtype=float32),
 array([-2.913703], dtype=float32))

In [45]:
def sigmoid_numpy(x):
    return 1/(1+np.exp(-x))

In [75]:
def log_loss(y_true,y_predicted):
    epsilon = 1e-15
    y_predicted_new = [ max(epsilon,i) for i in y_predicted]
    y_predicted_new = [ min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [105]:
def gradient(age,affordibility,y_true,epochs):
    w1=w2=1
    b = 0
    n = len(age)
    rate = 1.5
    for i in range(epochs):
        weighted_sum = w1*age + w2*affordibility + b
        y_predicted = sigmoid_numpy(weighted_sum)
        y_predicted = np.array(y_predicted)
        loss = log_loss(y_true,y_predicted)

        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted - y_true))
        w2d = (1/n)*np.dot(np.transpose(affordibility),(y_predicted - y_true))
        bd = np.mean(y_predicted-y_true)
        b = b - bd

        w1 = w1 - rate*w1d
        w2 = w2 - rate*w2d
        b = b - rate*bd

        print(f'epochs:{i},w1:{w1},w2:{w2},bias:{b},loss:{loss}')

    return w1,w2,b



In [107]:
gradient(x_train_scaled['age'],x_train_scaled['affordibility'],y_train,1000)

epochs:0,w1:0.924722900410531,w2:0.8450443761835869,bias:-0.5670933868184291,loss:0.7113403233723417
epochs:1,w1:0.936266178679654,w2:0.8275243250419814,bias:-0.7718482053343906,loss:0.6111898065572571
epochs:2,w1:0.9775473864132571,w2:0.8577701320747416,bias:-0.8528813929421891,loss:0.5993334601763546
epochs:3,w1:1.0248163531134664,w2:0.8955795092652117,bias:-0.9071998299336484,loss:0.5952356691757336
epochs:4,w1:1.0731134898827224,w2:0.9326123149107814,bias:-0.9550154601799028,loss:0.5916772135915024
epochs:5,w1:1.12150957529324,w2:0.9674760054689766,bias:-1.0001709322291448,loss:0.588344016054148
epochs:6,w1:1.1698231169071056,w2:1.0000590044429136,bias:-1.0434713891687781,loss:0.5852012896328987
epochs:7,w1:1.2180110279757594,w2:1.0304746034887218,bias:-1.0851558049415855,loss:0.5822270314461427
epochs:8,w1:1.2660567905279212,w2:1.0588677361186338,bias:-1.1253507780099365,loss:0.5794024602592492
epochs:9,w1:1.3139496090866996,w2:1.085379858429283,bias:-1.1641554229002784,loss:0.576

(10.6995275906339, 1.8595982888061473, -5.619156198365917)

In [108]:
#some how my model is working better than an tensorflow model
coef , intercept

(array([[5.060868],
        [1.40865 ]], dtype=float32),
 array([-2.913703], dtype=float32))